# Implement RNN code only with numpy

In [200]:
import numpy as np
import nltk
import datetime
import sys

## Generate Training Data
1. x and y label
2. For word forecast model, 0 stands for SENTENCE_START and 1 stands for SENTENCE_END

## example 1:
example sentense \
SENTENCE_START what do you think about language processing model, is it good? SENTENCE_END \
full sentence: [0, 51, 27, 16, 10, 856, 53, 25, 34, 69, 12, 13, 43, 41, 1] \
in this example, each number stands for a word, in real problem, normally a vector stands for a word, so the input is a series of vector 
(also can view as a 2-D array) \
input and output size: [time_steps, input_variables]

In [170]:
x_train = np.array([0, 51, 27, 16, 10, 856, 53, 25, 34, 69, 12, 13, 43, 41]).reshape(-1,1)
y_train = np.array([51, 27, 16, 10, 856, 53, 25, 34, 69, 12, 13, 43, 41,1]).reshape(-1,1)

## we have 6 words: I love go shopping with you, labels as:
[]

[]

## example 2:
we have 6 words and a sign: **I love going shopping with you.** labels as: \
I    ---------   [1,0,0,0,0,0,0] \
love   ---------  [0,1,0,0,0,0,0] \
going    ---------  [0,0,1,0,0,0,0] \
shopping --------- [0,0,0,1,0,0,0] \
with    --------- [0,0,0,0,1,0,1] \
you   ---------   [0,0,0,0,0,1,0] \
.    ---------   [0,0,0,0,0,0,1] \
and we also need begin and end mark, which is: \
BEGIN --------- [0,0,0,0,0,0,0] \
END --------- [1,1,1,1,1,1,1] \

To training our data, we make a few correct sentences by these words: \
Sentence 1: **I love going shopping with you.** \
Sentence 2: **I love you.**\
Sentence 3: **I love shopping**\
Sentense 4: **I love going with you**\

The Trainng Dataset is following:

In [171]:
words = {}
words['I'] = [1,0,0,0,0,0,0]
words['love'] = [0,1,0,0,0,0,0]
words['going'] = [0,0,1,0,0,0,0]
words['shopping'] = [0,0,0,1,0,0,0]
words['with'] = [0,0,0,0,1,0,0]
words['you'] = [0,0,0,0,0,1,0]
words['.'] = [0,0,0,0,0,0,1]
words['BEGIN'] = [0,0,0,0,0,0,0]
words['END'] = [1,1,1,1,1,1,1]

def generate_sentence_data(sentence):
    word_list = sentence.split()
    word_list.insert(0,'BEGIN')
    word_list.append('END')
    x_train = []
    y_train = []
    for i in range(0,len(word_list)-1):
        current_word = word_list[i]
        next_word = word_list[i+1]
        x_train.append(words[current_word])
        y_train.append(words[next_word])
    
    return np.array(x_train), np.array(y_train)

In [172]:
x_train, y_train = generate_sentence_data('I love going shopping with you .')

In [173]:
## example 3:
# word2vector


In [174]:
len(x_train)

8

In [175]:
y_train.shape

(8, 7)

## Process
1. Init stucture and parameters 
2. Forward 
    - Simple RNN
    - LSTM
3. Backpropogation
3. Update parameters
4. Predict


# My implement is a simple RNN with only one hiden layer


remember the forward propagation follow the function: \

$$ s_t =tanh(U^{(sx)}x_t + Ws_{t-1}^{(ss)})$$        
$$ \omicron = softmax(V_{S_t})$$       
lost function: $$J^{(t)}(\theta) = \sum_{i=1}^{|V|}(y_{t_i}^{'}logy_{t_i})$$   
## init function parameter here: 
size of input data (word dimension): x \
size of previous state (size of output dimension): s \
W(ss),U(sx),V(xs)

In this example, the default model is a three layers RNN model with a recurrent hiden layer, and an input, an output layer \
input dimension is **word_dim** \
output dimension is **d** \
number of hiden nodes is **hiden_dim** 

In [176]:
def softmax(x):
    e_x = np.exp(x - np.max(x))

    return e_x / e_x.sum()


## RNN model
[train](#Train-model)

In [220]:
class RNN():
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim,  hidden_dim))
        
## input x and output y (a full list through time s--> states   o--> output)
## input follow function 1,2
    def forward_propagation(self, x):
        # The total number of time steps
        if self.word_dim == 1:
            T = len(x)
        else:
            T = x.shape[0]
        
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):

            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh((self.U.dot(x[t].reshape(-1,1)) + self.W.dot(s[t-1]).reshape(-1,1))).reshape(self.hidden_dim)
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
 

    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
## lost function follow function 3    
## It's a cross entropy lost function that calculate classify problem
    def calculate_total_loss(self, x, y):
        L = 0
        # For each sentence...
        for i in range(0,len(y)):
#             print('this is %d th step'%(i))
#             print('input training data is',x[:i+1,:].reshape(-1,x.shape[1]))
            o, s = self.forward_propagation(x[:i+1,:].reshape(-1,x.shape[1]))
#             print('output result is:',o)
#             print('status result is: ',s)
            # We only care about our prediction of the &quot;correct&quot; words
            true_answer = y[i]
            predict_answer = o[i]
            correct_word_predictions = []
            for i in range(0,len(true_answer)):
                if true_answer[i] == 1:
                    correct_word_predictions.append(predict_answer[i])
                else:
                    correct_word_predictions.append(1)
            # Add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    def calculate_loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y)) 
        return self.calculate_total_loss(x,y)/N
    

    def bptt(self, x, y):
        if self.word_dim == 1:
            T = len(x)
        else:
            T = x.shape[0]
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
#        print('dLdU shape:', self.U.shape, 'dLdV shape:', self.V.shape, 'dLdW shape:', self.W.shape)
        ## delta_o is the output error, o is the forwardprop value (0-1)
        delta_o = o - y
#        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T-1)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
#            print('delta_o shape: ', delta_o.shape, ' delta_t shape', delta_t.shape)
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print &quot;Backpropagation step t=%d bptt step=%d &quot; % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])  
#                print('calculate columns: ',x[bptt_step], 'value ', dLdU[:,x[bptt_step]], 'shape: ',dLdU[:,x[bptt_step]].shape)
                dLdU[:,bptt_step] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW] 
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        # Calculate the gradients using backpropagation. We want to checker if these are correct.
        bptt_gradients = self.bptt(x, y)
        # List of all parameters we want to check.
        model_parameters = ['U', 'V', 'W']
        # Gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # Get the actual parameter value from the mode, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print('Performing gradient check for parameter %s with size %d.'%(pname, np.prod(parameter.shape)))
            # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # Save the original value so we can reset it later
                original_value = parameter[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                parameter[ix] = original_value + h
                gradplus = self.calculate_total_loss([x],[y])
                parameter[ix] = original_value - h
                gradminus = self.calculate_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # Reset parameter to original value
                parameter[ix] = original_value
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate The relative error: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # If the error is to large fail the gradient check
                if relative_error > error_threshold:
                    print('Gradient Check ERROR: parameter=%s ix=%s'% (pname, ix))
                    print(';+h Loss: %f'% gradplus)
                    print(';-h Loss: %f'% gradminus)
                    print(';Estimated_gradient: %f' % estimated_gradient)
                    print(';Backpropagation gradient: %f'% backprop_gradient)
                    print(';Relative Error: %f'% relative_error)
                    return it.iternext()
            print('Gradient check for parameter %s passed.'% (pname))
            
    # Performs one step of SGD.
    def numpy_sgd_step(self, x, y, learning_rate):
        # Calculate the gradients
        dLdU, dLdV, dLdW = self.bptt(x, y)
        # Change parameters according to gradients and learning rate
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW

## bptt
from the previous function, we know:\
$$ \frac{\partial E_3}{\partial V} = \frac{\partial E_3}{\partial \hat{y_3}} \frac{\partial \hat{y_3}}{\partial V} 
        = (\hat{y_3}-y_3) * s_3$$
$$\frac{\partial E_3}{\partial W} = \sum^3_{k=0}
\frac{\partial E_3}{\partial \hat{y_3}} \frac{\partial \hat{y_3}}{\partial s_3} \frac{\partial s_3}{\partial s_k} \frac{\partial s_k}{\partial W}$$
To calculate from bptt, we want to get: \
$$\frac{\partial L}{\partial W} = \sum_t \frac{\partial L}{\partial O}\frac{\partial O}{\partial s_t}\frac{\partial s_t}{\partial W}
 = \sum_t \sum_{k=1}^{t} \frac{\partial L}{\partial O}\frac{\partial O}{\partial s_t}\frac{\partial s_t}{\partial s_k}\frac{\partial s_k}{\partial W}$$
$$\frac{\partial L}{\partial V} = \sum_{t=0}^T \partial O* s_t^T $$ 
$$\frac{\partial L}{\partial U} = \sum_t \frac{\partial L}{\partial O}\frac{\partial 0}{\partial s_t}\frac{\partial s_t}{\partial U}$$
And update U,V,W by gradient descent:\
$$ U_{n} = U_{n-1} - \eta * \frac{\partial L}{\partial U_{n-1}}$$
$$ V_{n} = V_{n-1} - \eta * \frac{\partial L}{\partial V_{n-1}}$$
$$ W_{n} = W_{n-1} - \eta * \frac{\partial L}{\partial W_{n-1}}$$

This [blog](https://medium.com/@aidangomez/let-s-do-this-f9b699de31d9) and this [paper](https://www.researchgate.net/publication/308980601_A_Gentle_Tutorial_of_Recurrent_Neural_Network_with_Error_Backpropagation) illustrate backpropagation clearly

$$\mathscr{L}$$

## Gradient Checking

verify implement is corrent:\
$$ \frac{\partial L}{\partial \theta} L \sim \lim_{h-> 1} \frac{J(\theta + h)- J(\theta - h)}{2h}$$

## Train model
[Model](#RNN-model)

In [224]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ('%s: Loss  after num_examples_seen=%d epoch=%d: %f '% (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ('Setting learning rate to %f'%learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.numpy_sgd_step(X_train, y_train, learning_rate)
            num_examples_seen += 1

## Let's use example 2 sentence as the training sentence

In [225]:
## init model
model = RNN(x_train.shape[1], hidden_dim=10, bptt_truncate=4)

In [226]:
## test each module
o,s = model.forward_propagation(x_train)

In [227]:
model.calculate_loss(x_train, y_train)

F:\Anaconda\envs\ds\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0.4806791285061665

In [228]:
[dLdU, dLdV, dLdW] = model.bptt(x_train,y_train)

In [229]:
train_with_sgd(model, x_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5)

2020-06-16 20:37:30: Loss  after num_examples_seen=0 epoch=0: 0.480679 
2020-06-16 20:37:30: Loss  after num_examples_seen=40 epoch=5: 0.463726 
2020-06-16 20:37:30: Loss  after num_examples_seen=80 epoch=10: 0.445329 
2020-06-16 20:37:30: Loss  after num_examples_seen=120 epoch=15: 0.423961 
2020-06-16 20:37:30: Loss  after num_examples_seen=160 epoch=20: 0.400539 
2020-06-16 20:37:30: Loss  after num_examples_seen=200 epoch=25: 0.381503 
2020-06-16 20:37:30: Loss  after num_examples_seen=240 epoch=30: 0.368695 


F:\Anaconda\envs\ds\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


2020-06-16 20:37:30: Loss  after num_examples_seen=280 epoch=35: 0.359040 
2020-06-16 20:37:30: Loss  after num_examples_seen=320 epoch=40: 0.352621 
2020-06-16 20:37:30: Loss  after num_examples_seen=360 epoch=45: 0.350650 
2020-06-16 20:37:30: Loss  after num_examples_seen=400 epoch=50: 0.352442 
Setting learning rate to 0.002500
2020-06-16 20:37:30: Loss  after num_examples_seen=440 epoch=55: 0.354374 
Setting learning rate to 0.001250
2020-06-16 20:37:30: Loss  after num_examples_seen=480 epoch=60: 0.355543 
Setting learning rate to 0.000625
2020-06-16 20:37:30: Loss  after num_examples_seen=520 epoch=65: 0.356173 
Setting learning rate to 0.000313
2020-06-16 20:37:30: Loss  after num_examples_seen=560 epoch=70: 0.356498 
Setting learning rate to 0.000156
2020-06-16 20:37:30: Loss  after num_examples_seen=600 epoch=75: 0.356664 
Setting learning rate to 0.000078
2020-06-16 20:37:30: Loss  after num_examples_seen=640 epoch=80: 0.356747 
Setting learning rate to 0.000039
2020-06-16 2

### Job Done!!